### Julia arrays are not copied when assigned to another variable. After `A = B`, changing elements of `B` will modify `A` as well.

In [11]:
@show A = randn(2, 2)
B = A  # B is not a new copy of A, modify B will also modify A.
B[1, 1] += 1
@show B
@show A;

A = randn(2, 2) = [-0.707163 1.8556; -1.15064 -0.561265]
B = [0.292837 1.8556; -1.15064 -0.561265]
A = [0.292837 1.8556; -1.15064 -0.561265]


### Updating operators like `+=` do not operate in-place, they are equivalent to `A = A + B` which _**rebinds**_ the left-hand side to the result of the right-hand side expression.

In [12]:
B += ones(2, 2)  # += is not in-place, this will cause re-binds B to a new variable.
@show A
@show B;

A = [0.292837 1.8556; -1.15064 -0.561265]
B = [1.29284 2.8556; -0.150644 0.438735]


### Julia does not treat the numbers `0` and `1` as `Booleans`.
You **CANNOT** write `if (1)` in Julia, because `if` statements accept *ONLY* booleans. Instead, you can write `if true`, `if Bool(1)`, or `if 1==1`.

In [2]:
if 1
end

TypeError: TypeError: non-boolean (Int64) used in boolean context

### Unexpected memory allocation is almost always a sign of some problem with your code. Take such indications seriously.

1. Unexpected memory is usually a problem with type-stability or creating many small temporary arrays.
1. In addition to the allocation itself, it's _**very likely that the code generated for your function is far from optimal**_.

In [21]:
x = rand(1000)

function sum()
    s = 0
    for i in x
        s += i
    end
    return s
end
@time sum()
@time sum();  # Pay attention to allocation information!

  0.006590 seconds (5.67 k allocations: 177.454 KiB)
  0.000149 seconds (3.49 k allocations: 70.313 KiB)


In [22]:
function sum2(x)
    s = 0
    for i in x
        s += i
    end
    return s
end

@time sum2(x)  # On the first call this function gets compiled. 
@time sum2(x);

  0.015741 seconds (14.77 k allocations: 779.980 KiB)
  0.000004 seconds (5 allocations: 176 bytes)


In [26]:
function test()
  @time sum2(x)
end

test()  # very few memory allocation leading to much faster.

  0.000005 seconds (1 allocation: 16 bytes)


492.19434340617073

### Combinatorial explosion of types will be counterproductive

If `items[i+1]` has a different type than `item[i]`, Julia has to:
1. look up the type at run-time;
1. search for the appropriate method in method tables;
1. decide (via type intersection) which one matches;
1. determine whether it has been JIT-compiled yet (and do so if not);
1. and then make the call.

From Julia's doc:
> In essence, you're asking the full type-system and JIT-compilation machinery to basically execute the equivalent of a switch statement or dictionary lookup in your own code. Some run-time benchmarks comparing (1) type dispatch, (2) dictionary lookup, and (3) a "switch" statement can be found on the [mailing list](https://groups.google.com/forum/#!msg/julia-users/jUMu9A3QKQQ/qjgVWr7vAwAJ).

### Julia's compiler specializes code for argument types at function boundaries.